# Task 1:

## Part-A

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import torch
from torch.utils.data import DataLoader
from transformers import BertForSequenceClassification, BertTokenizer
import pandas as pd

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=1)
model = model.from_pretrained("/content/drive/MyDrive/NLP_A3/Task1A/NLP_A3_Model_A-20240330T110255Z-001/NLP_A3_Model_A")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import torch
from torch.utils.data import Dataset
from transformers import BertTokenizer


test_data = pd.read_csv("/content/sample_test.csv", delimiter='\t')
# print(test_data)
test_data.columns = ['id', 'sentence1', 'sentence2']
test_data.to_csv('/content/sample_test.csv', sep="\t",index=False)


class CustomDatasett(Dataset):
    def __init__(self, file_path):
        self.data = pd.read_csv(file_path, delimiter='\t')
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        # self.sent1_max = 0
        # self.sent2_max = 0

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data.iloc[idx]
        sentence1 = ( sample["sentence1"] if type(sample["sentence1"]) !=float  else "" )
        sentence2 = ( sample["sentence2"] if type(sample["sentence2"]) !=float  else "" )
        # self.sent1_max = max(self.sent1_max,len(sentence1.split()))
        # self.sent2_max = max(self.sent2_max,len(sentence2.split()))
        max_len = 512
        combined_sentence = sentence1 + "[SEP]" + sentence2
        print(combined_sentence)
        segment_id = torch.hstack([torch.zeros((1,len(sentence1.split())+2)),torch.ones((1,len(sentence2.split())+1)),torch.zeros((1,max_len-(len(sentence1.split())+2)-(len(sentence2.split())+1)))]).long()

        encoded_input = self.tokenizer(combined_sentence, return_tensors='pt',padding ="max_length",max_length=max_len,truncation=True)

        encoded_input['token_type_ids'] = segment_id



        # Preprocessing, tokenization, or any other operations can be performed here
        # For simplicity, let's just return the text and label as tensors
        return encoded_input["input_ids"], encoded_input["token_type_ids"] ,encoded_input["attention_mask"]


# Example usage:

file_path = "/content/sample_test.csv"
test_custom_dataset = CustomDatasett(file_path)
model.eval()
scores = []
for i,j,k in test_custom_dataset:
  print((model(input_ids=i, token_type_ids=j, attention_mask=k)))
  scores.append( (model(input_ids=i, token_type_ids=j, attention_mask=k).logits[0][0]*5).item())

test_data['score'] = scores
# Predict whether the two sentences belong to the same category or not
test_data = test_data[['id', 'score', 'sentence1', 'sentence2']]
test_data.to_csv('/content/sample_test_out.csv', sep="\t",index=False)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

A man with a hard hat is dancing.[SEP]A man wearing a hard hat is dancing.
SequenceClassifierOutput(loss=None, logits=tensor([[1.0707]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
A young child is riding a horse.[SEP]A child is riding a horse.
SequenceClassifierOutput(loss=None, logits=tensor([[1.0217]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
A man is feeding a mouse to a snake.[SEP]The man is feeding a mouse to the snake.
SequenceClassifierOutput(loss=None, logits=tensor([[1.0344]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
A woman is playing the guitar.[SEP]A man is playing guitar.
SequenceClassifierOutput(loss=None, logits=tensor([[0.5286]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
A woman is playing the flute.[SEP]A man is playing a flute.
SequenceClassifierOutput(loss=None, logits=tensor([[0.6492]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
A woman is cutting an onion.[

## Part-C

In [1]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 1.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [3]:
from sentence_transformers import SentenceTransformer, InputExample, losses
import importlib
from torch.utils.data import DataLoader
from sentence_transformers import evaluation
import torch
from torch.utils.data import Dataset
from transformers import BertTokenizer
import numpy as np

In [7]:
from sentence_transformers import SentenceTransformer, InputExample, losses
import importlib
from torch.utils.data import DataLoader
from sentence_transformers import evaluation
import torch
from torch.utils.data import Dataset
from transformers import BertTokenizer
import numpy as np
test_data = pd.read_csv("/content/test_group_52.csv", delimiter='\t')

test_data.columns = ['id', 'sentence1', 'sentence2']
test_data.to_csv('/content/test_group_52.csv', sep="\t",index=False)


class CustomDatasett(Dataset):
    def __init__(self, file_path):
        self.data = pd.read_csv(file_path, delimiter='\t')
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        # self.sent1_max = 0
        # self.sent2_max = 0
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data.iloc[idx]
        sentence1 = ( sample["sentence1"] if type(sample["sentence1"]) !=float  else "" )
        sentence2 = ( sample["sentence2"] if type(sample["sentence2"]) !=float  else "" )
        # score = sample['score']
        # self.sent1_max = max(self.sent1_max,len(sentence1.split()))
        # self.sent2_max = max(self.sent2_max,len(sentence2.split()))
        # max_score = 5.0
        return sentence1,sentence2#, float(score/max_score)
file_path = "/content/test_group_52.csv"
test_custom_dataset = CustomDatasett(file_path)

tsentences1 = [ ]
tsentences2 = [ ]
for sent1,sent2 in test_custom_dataset:
  tsentences1.append(sent1)
  tsentences2.append(sent2)

# Initialize the CrossEncoder model
tmodel = SentenceTransformer('/content/drive/MyDrive/NLP_A3/Task1BC/mini-gun-1-20240330T072011Z-001/mini-gun-1/mini_model')

a = tmodel.encode(tsentences1)
b = tmodel.encode(tsentences2)


# Assuming A and B are numpy arrays
A = a
B = b
# Compute cosine similarity
dot_product = np.sum(A * B, axis=1)
norm_A = np.linalg.norm(A, axis=1)
norm_B = np.linalg.norm(B, axis=1)
cosine_similarities = dot_product / (norm_A * norm_B)

def scale_cosine_similarity(cosine_similarity):
    scaled_similarity = ((cosine_similarity + 1) / 2 )* 5
    return scaled_similarity
new_scale_score =scale_cosine_similarity(cosine_similarities)

print("Scale_score Cosine Similarities:")
print(new_scale_score)


# Predict whether the two sentences belong to the same category or not


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Scale_score Cosine Similarities:
[4.0570827 3.2267025 4.592778  ... 3.263517  3.651514  4.2222586]


In [8]:
test_data['score'] = new_scale_score
test_data = test_data[['id', 'score', 'sentence1', 'sentence2']]

test_data.to_csv('/content/sample_test_out.csv', sep="\t",index=False)


# test_data = pd.read_csv("/content/sample_test.csv", delimiter='\t')

## Part-B

In [ ]:
!pip install sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer, InputExample, losses
import importlib
from torch.utils.data import DataLoader
from sentence_transformers import evaluation
import torch
from torch.utils.data import Dataset
from transformers import BertTokenizer
import numpy as np

In [ ]:
from sentence_transformers import SentenceTransformer, InputExample, losses
import importlib
from torch.utils.data import DataLoader
from sentence_transformers import evaluation
import torch
from torch.utils.data import Dataset
from transformers import BertTokenizer
import numpy as np
test_data = pd.read_csv("/content/sample_test.csv", delimiter='\t')

test_data.columns = ['id', 'sentence1', 'sentence2']
test_data.to_csv('/content/sample_test.csv', sep="\t",index=False)


class CustomDatasett(Dataset):
    def __init__(self, file_path):
        self.data = pd.read_csv(file_path, delimiter='\t')
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        # self.sent1_max = 0
        # self.sent2_max = 0
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data.iloc[idx]
        sentence1 = ( sample["sentence1"] if type(sample["sentence1"]) !=float  else "" )
        sentence2 = ( sample["sentence2"] if type(sample["sentence2"]) !=float  else "" )
        # score = sample['score']
        # self.sent1_max = max(self.sent1_max,len(sentence1.split()))
        # self.sent2_max = max(self.sent2_max,len(sentence2.split()))
        # max_score = 5.0
        return sentence1,sentence2#, float(score/max_score)
file_path = "/content/sample_test.csv"
test_custom_dataset = CustomDatasett(file_path)

tsentences1 = [ ]
tsentences2 = [ ]
for sent1,sent2 in test_custom_dataset:
  tsentences1.append(sent1)
  tsentences2.append(sent2)

# Initialize the CrossEncoder model
tmodel = SentenceTransformer('all-MiniLM-L6-v2')

a = tmodel.encode(tsentences1)
b = tmodel.encode(tsentences2)


# Assuming A and B are numpy arrays
A = a
B = b
# Compute cosine similarity
dot_product = np.sum(A * B, axis=1)
norm_A = np.linalg.norm(A, axis=1)
norm_B = np.linalg.norm(B, axis=1)
cosine_similarities = dot_product / (norm_A * norm_B)

def scale_cosine_similarity(cosine_similarity):
    scaled_similarity = (cosine_similarity + 1) / 2 * 5
    return scaled_similarity
new_scale_score =scale_cosine_similarity(cosine_similarities)

print("Scale_score Cosine Similarities:")
print(new_scale_score)


# Predict whether the two sentences belong to the same category or not


Scale_score Cosine Similarities:
[4.983429  4.884906  4.9502535 4.1177073 4.3655686 4.386547 ]


In [10]:
test_data

,id,score,sentence1,sentence2
0,0,4.057083,A woman is slicing garlics.,A woman is slicing an onion.
1,1,3.226702,A man is kicking pots of water.,A man is turning foods.
2,2,4.592778,A man wearing black formal wear & a black top ...,A man with a top hat riding a white horse.
3,3,3.086869,Two people are in bed.,Two boxers are in the ring.
4,4,3.611768,Two ducks are standing by the water.,The birds are swimming in the water.
...,...,...,...,...
1113,1113,4.770375,Hormone replacement therapy that combines estr...,A major British study has added to the evidenc...
1114,1114,4.500027,The polar bears fought over the kill.,Polar bears are fighting each other.
1115,1115,3.263517,A woman is slicing an onion.,A woman is slicing lemons.
1116,1116,3.651514,There are a few things I think you should do.,There are a few things I don't understand:


In [11]:
test_data['score'] = new_scale_score
test_data = test_data[['id', 'score', 'sentence1', 'sentence2']]

test_data.to_csv('/content/sample_test_out.csv',index=False)


# test_data = pd.read_csv("/content/sample_test.csv", delimiter='\t')

# Task 2:

## Part-A

In [ ]:
!pip install datasets
!pip install -U torchdata
!pip install -U spacy
!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_sm
!pip install bert_score
!pip install --index-url https://test.pypi.org/simple/ pymeteor

from nltk.translate.bleu_score import sentence_bleu
import torch
import pymeteor.pymeteor as pymeteor
from bert_score import BERTScorer
from datasets import load_dataset
from nltk.translate.bleu_score import sentence_bleu
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from typing import Iterable, List
import matplotlib.pyplot as plt
from torchtext.data.metrics import bleu_score
from nltk.translate import meteor_score
from bert_score import score
from torch import Tensor
import torch.nn as nn
from torch.nn import Transformer
import math
import pandas as pd
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from tqdm import tqdm
from torchdata.datapipes.iter import ShardingFilter
from nltk.translate.bleu_score import SmoothingFunction

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 30.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 33.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.9 MB/s eta 0:00:00
Looking in indexes: https://test.pypi.org/simple/


In [ ]:
# load training data
train_data = load_dataset("wmt16", "de-en", split="train[:50000]")
val_data = load_dataset("wmt16", "de-en", split="validation")

# Few examples
print("Training Examples:")
print(train_data[0])
print("\nValidation Examples:")
print(val_data[0])

train_data_final = []
for data in train_data['translation']:
    train_data_final.append((data['de'], data['en']))

val_data_final = []
for data in val_data['translation']:
    val_data_final.append((data['de'], data['en']))

train_data_final = ShardingFilter(train_data_final)
val_data_final = ShardingFilter(val_data_final)

SRC_LANGUAGE = 'de'
TGT_LANGUAGE = 'en'

# token and vocab transformations
token_transform = {}
vocab_transform = {}

# Tokenizers for German and English
token_transform['de'] = get_tokenizer('spacy', language='de_core_news_sm')
token_transform['en'] = get_tokenizer('spacy', language='en_core_web_sm')

def yield_tokens(data_iter: Iterable, language: str) -> List[str]:
    language_index = {SRC_LANGUAGE: 0, TGT_LANGUAGE: 1}

    for data_sample in data_iter:
        yield token_transform[language](data_sample[language_index[language]])

#special symbols and indices
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

# torchtext's Vocab object for both languages
for ln in ['de', 'en']:
    vocab_transform[ln] = build_vocab_from_iterator(yield_tokens(train_data_final, ln),
                                                    min_freq=1,
                                                    specials=special_symbols,
                                                    special_first=True)

# Set ``UNK_IDX`` as the default index
for ln in ['de', 'en']:
    vocab_transform[ln].set_default_index(UNK_IDX)


DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# helper Module that adds positional encoding to the token embedding.
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# helper Module to convert tensor of input indices into corresponding tensor of token embeddings
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

# Seq2Seq Network
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = Transformer(d_model=emb_size,
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)


def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

torch.manual_seed(0)

SRC_VOCAB_SIZE = len(vocab_transform[SRC_LANGUAGE])
TGT_VOCAB_SIZE = len(vocab_transform[TGT_LANGUAGE])
EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 512
BATCH_SIZE = 64
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3

transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)

for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer = transformer.to(DEVICE)
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)
optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

from torch.nn.utils.rnn import pad_sequence

# helper function to club together sequential operations
def sequential_transforms(*transforms):
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func

# function to add BOS/EOS and create tensor for input sequence indices
def tensor_transform(token_ids: List[int]):
    return torch.cat((torch.tensor([BOS_IDX]),
                      torch.tensor(token_ids),
                      torch.tensor([EOS_IDX])))

# ``src`` and ``tgt`` language text transforms to convert raw strings into tensors indices
text_transform = {}
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    text_transform[ln] = sequential_transforms(token_transform[ln], #Tokenization
                                               vocab_transform[ln], #Numericalization
                                               tensor_transform)


def collate_fn(batch):
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:
        src_batch.append(text_transform[SRC_LANGUAGE](src_sample.rstrip("\n")))
        tgt_batch.append(text_transform[TGT_LANGUAGE](tgt_sample.rstrip("\n")))

    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
    return src_batch, tgt_batch

def train_epoch(model, optimizer):
    model.train()
    losses = 0
    train_iter = train_data_final

    train_dataloader = DataLoader(train_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)
    train_dataloader = tqdm(train_dataloader, desc="Training", leave=False)
    for src, tgt in train_dataloader:
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)
        tgt_input = tgt[:-1, :]
        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)
        logits = model(src, tgt_input, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask, src_padding_mask)
        optimizer.zero_grad()
        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()
        optimizer.step()
        losses += loss.item()
    return losses / len(list(train_dataloader))


def evaluate(model):
    model.eval()
    losses = 0
    val_iter = val_data_final

    val_dataloader = DataLoader(val_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)

    for src, tgt in val_dataloader:
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)
        tgt_input = tgt[:-1, :]
        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)
        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)
        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        losses += loss.item()
    return losses / len(list(val_dataloader))

def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)
    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len-1):
        memory = memory.to(DEVICE)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                    .type(torch.bool)).to(DEVICE)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()
        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
            break
    return ys


# actual function to translate input sentence into target language
def translate(model: torch.nn.Module, src_sentence: str):
    model.eval()
    src = text_transform[SRC_LANGUAGE](src_sentence).view(-1, 1)
    num_tokens = src.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode(
        model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()
    return " ".join(vocab_transform[TGT_LANGUAGE].lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<bos>", "").replace("<eos>", "")


Generating train split:   0%|          | 0/4548885 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2169 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2999 [00:00<?, ? examples/s]

Training Examples:
{'translation': {'de': 'Wiederaufnahme der Sitzungsperiode', 'en': 'Resumption of the session'}}

Validation Examples:
{'translation': {'de': 'Die Premierminister Indiens und Japans trafen sich in Tokio.', 'en': 'India and Japan prime ministers meet in Tokyo'}}


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [ ]:
# Inference Pipeline for Testing Model
def inference_pipeline(input_csv_path, output_csv_path):
    model_path = '/content/drive/MyDrive/NLP_A3/Task2A/German_to_English_model.pth'
    model = transformer
    model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

    df = pd.read_csv(input_csv_path)
    translations = [translate(transformer, text) for text in df['de']]
    df['en'] = translations
    df.to_csv(output_csv_path, index=False)

inference_pipeline('/content/drive/MyDrive/NLP_A3/Task2A/input.csv', '/content/drive/MyDrive/NLP_A3/Task2A/output.csv')

## Part-B

In [ ]:
import pandas as pd
pd.DataFrame({"en":['this is my name']})

,en
0,this is my name


In [ ]:
# Inference Pipeline for Testing Data
import pandas as pd
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load pre-trained T5 model and tokenizer
model_name = "t5-small"
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)

# Function to translate a single English sentence to German
def translate_sentence(english_sentence):
    try:
        print("Input English sentence:", english_sentence)
        # Tokenize input text
        inputs = tokenizer.encode("translate English to German: " + english_sentence, return_tensors="pt", max_length=512, truncation=True)

        # Generate translation
        with torch.no_grad():
            outputs = model.generate(inputs, max_length=100, num_beams=4, early_stopping=True)

        # Decode the generated translation
        translated_sentence = tokenizer.decode(outputs[0], skip_special_tokens=True)
        print("Translated German sentence:", translated_sentence)
        return translated_sentence
    except Exception as e:
        print("Error during translation:", e)
        return ""

# Function to translate English sentences in a CSV file and output translated CSV
def translate_csv(input_csv_path, output_csv_path):
    try:
        # Read input CSV
        df = pd.read_csv(input_csv_path)

        # Translate English sentences
        df['de'] = df['en'].apply(translate_sentence)

        # Save translated CSV
        df.to_csv(output_csv_path, index=False)

        print(f"Translation completed. Translated CSV saved to {output_csv_path}.")
    except Exception as e:
        print("Error during translation:", e)

# Run translation pipeline for CSV input and output
input_csv_path = "input.csv"
output_csv_path = "output.csv"
translate_csv(input_csv_path, output_csv_path)


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Error during translation: [Errno 2] No such file or directory: 'input.csv'


## Part-C

In [12]:
import os
os.chdir("/content/drive/MyDrive/NLP_A3/Task2C")
os.getcwd()

'/content/drive/.shortcut-targets-by-id/1jagc82ImUO0B7jTE6LrEgFwtap8w18YC/A3_52/Task2C'

In [ ]:
!unzip "/content/drive/MyDrive/NLP_A3/Task2C/t5_fine_tunning_model.zip"

unzip:  cannot find or open /content/drive/MyDrive/NLP_A3/Task2C/t5_fine_tunning_model.zip, /content/drive/MyDrive/NLP_A3/Task2C/t5_fine_tunning_model.zip.zip or /content/drive/MyDrive/NLP_A3/Task2C/t5_fine_tunning_model.zip.ZIP.


In [13]:
from transformers import TFAutoModelForSeq2SeqLM, AutoTokenizer

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/NLP_A3/Task2C")
model = TFAutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/NLP_A3/Task2C")


All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [14]:
import pandas as pd

# Read CSV file
# df = pd.read_csv("/content/drive/MyDrive/NLP_A3/Task2C/sentences.csv")

# Assuming your German text column is named 'German'
german_texts = "Mal sehen, wie gut Sie diese Aufgabe gemeistert haben"

# Tokenize German input texts
german_inputs = tokenizer(german_texts, return_tensors="tf", padding=True, truncation=True)

# Generate translations for German input texts
translated = model.generate(input_ids=german_inputs['input_ids'], attention_mask=german_inputs['attention_mask'])

# Decode generated translations
decoded_translations = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]

# Create a DataFrame with German sentences and their corresponding English translations
translation_df = pd.DataFrame({'de': german_texts, 'en': decoded_translations})

# Save the DataFrame to a CSV file
translation_df.to_csv("/content/drive/MyDrive/NLP_A3/Task2C/translations.csv", index=False)

# Print a message indicating successful save
print("Translations saved to translations.csv")


/usr/local/lib/python3.10/dist-packages/transformers/generation/tf_utils.py:837: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length.  recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Translations saved to translations.csv


In [15]:
pd.read_csv('/content/drive/MyDrive/NLP_A3/Task2C/translations.csv')

,de,en
0,"Mal sehen, wie gut Sie diese Aufgabe gemeister...","Mal, you see how well you have managed this task."
